In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from mlxtend.preprocessing import TransactionEncoder
!pip install mlxtend==0.23.1


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [8]:
# load the data set ans show the first five transaction
df = pd.read_csv(r"https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv")
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [9]:
unique = set(np.ravel(df))
for i in df:
  unique.update(df[i].unique())
print(unique)

{'Eggs', 'Bread', 'Meat', 'Wine', 'Pencil', 'Diaper', 'Milk', 'Bagel', nan, 'Cheese'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [10]:
#create an itemset based on the products
itemset = set(unique)

# encoding the feature
encodedValue = []
for index, row in df.iterrows():
    rowset = set(row) 
    items = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for i in uncommons:
        items[i] = 0
    for j in commons:
        items[j] = 1
    encodedValue.append(items)

items

{'Pencil': 0,
 'Diaper': 0,
 'Milk': 0,
 'Cheese': 0,
 'Eggs': 1,
 'Bread': 1,
 'Meat': 1,
 'Wine': 1,
 'Bagel': 1,
 nan: 1}

In [11]:
  # create new dataframe from the encoded features
new_df = pd.DataFrame(encodedValue)
  # show the new dataframe
new_df.head()

,NaN,Milk,Bagel,Eggs,Bread,Meat,Wine,Pencil,Diaper,Cheese
0,0,0,0,1,1,1,1,1,1,1
1,0,1,0,0,1,1,1,1,1,1
2,1,1,0,1,0,1,1,0,0,1
3,1,1,0,1,0,1,1,0,0,1
4,1,0,0,0,0,1,1,1,0,0


In [12]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
new_df = new_df.drop(new_df.columns[1], axis=1)
new_df.head()

,NaN,Bagel,Eggs,Bread,Meat,Wine,Pencil,Diaper,Cheese
0,0,0,1,1,1,1,1,1,1
1,0,0,0,1,1,1,1,1,1
2,1,0,1,0,1,1,0,0,1
3,1,0,1,0,1,1,0,0,1
4,1,0,0,0,1,1,1,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [13]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules

freqpurchase = apriori(new_df, min_support=0.2, use_colnames=True)
freqpurchase.head()

c:\Users\bryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.869841,(nan)
1,0.425397,(Bagel)
2,0.438095,(Eggs)
3,0.504762,(Bread)
4,0.476190,(Meat)


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [14]:
assRules = association_rules(freqpurchase, metric="confidence", min_threshold=0.6)
assRules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Bagel),(nan),0.425397,0.869841,0.336508,0.791045,0.909413,-0.033520,0.622902,-0.147743
1,(Eggs),(nan),0.438095,0.869841,0.336508,0.768116,0.883053,-0.044565,0.561310,-0.190735
2,(Bread),(nan),0.504762,0.869841,0.396825,0.786164,0.903801,-0.042237,0.608683,-0.176903
3,(Meat),(nan),0.476190,0.869841,0.368254,0.773333,0.889051,-0.045956,0.574230,-0.192405
4,(Wine),(nan),0.438095,0.869841,0.317460,0.724638,0.833069,-0.063613,0.472682,-0.262869


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

Antecedent Support: The frequency at which the "if" component of the rule appears in transactions. Higher values suggest that the items are commonly purchased.

Consequent Support: The frequency at which the "then" component of the rule appears in transactions. Higher values indicate that these items are also commonly purchased.

Support: The ratio of transactions that include both the antecedent and consequent. Higher support values signify stronger associations between the items.

Confidence: The likelihood of the consequent occurring when the antecedent is present. Higher confidence indicates that the rule is more dependable.

Lift: A measure of how much more probable the consequent is when the antecedent is present compared to its random occurrence. A lift value greater than 1 suggests a positive relationship.

Leverage: The difference between the actual frequency of the antecedent and consequent appearing together and the frequency expected if they were independent. Positive leverage values indicate a stronger association.

Conviction: A measure of how much the occurrence of the consequent relies on the antecedent. Higher conviction values imply a stronger dependency.